## Odstranitev osamelcev in normalizacija atributov

In [117]:
import pandas as pd

In [118]:
df_tracks = pd.read_csv('Data\Playlist\Playlists.csv', index_col=0)
df_tracks.drop_duplicates(subset=['id'], inplace=True)
df_tracks.set_index("id", inplace=True)

In [119]:
df_tracks.shape

(2011, 20)

In [120]:
genres = ['pop', 'rock', 'rap', 'edm', 'latin', 'classical']

In [121]:
att = ['speechiness', 'danceability', 'energy', 
        'valence', 'acousticness', 'liveness',
        'instrumentalness', 'tempo', 'loudness',
        'duration_ms']

In [122]:
print(att[9])

duration_ms


In [123]:
def remove_outliers_df(att, df_base):
    df = df_base[att]
    
    df = remove_outliers(df)
    
    new_df = df_base.drop(att, axis=1)
    new_df = pd.merge(df, new_df, on='id', how='inner')
    
    return new_df

def remove_outliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 

    filter = (df >= Q1 - 10 * IQR) & (df <= Q3 + 10 *IQR)
    return df.loc[filter]  

Nastavimo normalno mejo za cas trajnja pesmi, kar je izven meje (vec kot 20 min in mknj kot 50s) odstranimo

In [176]:
new_df = df_tracks[df_tracks['duration_ms'] > 52800]  
new_df = new_df[new_df['duration_ms'] < 1280000]  

In [177]:
print(df_tracks.shape[0] - new_df.shape[0])

2


In [178]:
new_df['duration_ms'].min()

52929

In [179]:
new_df['duration_ms'].max()

1271947

In [180]:
def normalize(att, df):
    # normaliziramo vrednosti 
    normalized_df=(df[att]-df[att].min())/(df[att].max()-df[att].min())
    new_df = df.drop(att, axis=1)
    new_df = pd.merge(normalized_df, new_df, on='id', how='inner')
    return new_df

In [181]:
new_df['loudness'] = new_df['loudness'] + 60
new_df = normalize('loudness', new_df)
new_df = normalize('tempo', new_df)
new_df = normalize('duration_ms', new_df)

In [182]:
new_df.head(5)

,duration_ms,tempo,loudness,name,artist_id,artist,album_id,album,popularity,playlist,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature
id,,,,,,,,,,,,,,,,,,,,
17i5jLpzndlQhbS4SrTd0B,0.081501,0.343939,0.578072,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",5aIqB5nVVvmFsvSdExz408,Johann Sebastian Bach,3cvjMNWg7t46NyDZzujfNg,"Bach: Cello Suites Nos. 1, 5 & 6",65,0,0.2750,0.1570,7,1,0.0636,0.890,0.842,0.186,0.3040,4
4SFBV7SRNG2e2kyL1F6kjU,0.070859,0.627115,0.423640,"The Well-Tempered Clavier: Book 1, BWV 846-869...",5aIqB5nVVvmFsvSdExz408,Johann Sebastian Bach,1XORY4rQNhqkZxTze6Px90,Piano Book (Deluxe Edition),58,0,0.2210,0.1260,0,1,0.0447,0.989,0.897,0.102,0.2160,4
3zLTPuucd3e6TxZnu2dlVS,0.130546,0.589478,0.299563,"Bagatelle No. 25 in A Minor, WoO 59 ""Für Elise""",2wOqMjp9TyABvtHdOSOTUS,Ludwig van Beethoven,230jpjk9w7MGupNMMvXoaN,"Für Elise, Bagatelle No. 25 in A Minor, WoO 59",58,0,0.2890,0.0306,9,0,0.0446,0.987,0.911,0.102,0.1180,3
7h6GoPvGHC9uzZJ8bNvfIq,0.256123,0.374500,0.380955,"Suite No. 3 in D Major, BWV 1068: II. Air",5aIqB5nVVvmFsvSdExz408,Johann Sebastian Bach,1WuKnZDy27KDdLr96p0D8A,Albinoni: Adagio in G minor / Pachelbel: Canon,57,0,0.0753,0.0700,2,1,0.0440,0.918,0.947,0.146,0.0625,4
5x3TUfYzgZIdt0Kaop64ss,0.204397,0.399044,0.638688,Pachelbel: Canon and Gigue in D Major for 3 Vi...,62TD7509VQIxUe4WpwO0s3,Johann Pachelbel,35AZkwcf7WxuzNQ6iOhiIh,Relax,0,0,0.1300,0.1580,2,1,0.0350,0.748,0.924,0.100,0.0998,4


In [183]:
new_df['duration_ms'].mean()

0.15639706977712822

Odstranjevanje osamelcev, normalizacija za datoteko Playlists5.csv.

In [185]:
# new_df.to_csv('Data\Playlist\Playlists5.csv')

In [197]:
df_tracks1 = pd.read_csv('Data\Genre\Tracks_2_AudioF.csv', index_col=(0,1))
df_tracks1.drop_duplicates(subset=['id'], inplace=True)
df_tracks1.set_index("id", inplace=True)

df_tracks1['loudness'] = df_tracks1['loudness'] + 60
df_tracks1 = normalize('loudness', df_tracks1)
df_tracks1 = normalize('tempo', df_tracks1)
df_tracks1 = normalize('duration_ms', df_tracks1)

In [198]:
df_tracks1.head()

,duration_ms,tempo,loudness,name,artist_id,genre,album_id,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature
id,,,,,,,,,,,,,,,,,
2HSmyk2qMN8WQjuGhaQgCk,0.069475,0.349686,0.850311,Champagne Poetry,3TVXtAsR1Inumwj472S9r4,rap,3SpBlxme9WbeQdI9kx7KAV,0.423,0.667,6,0,0.326,0.758,0.000000,0.246,0.496,4
6jy9yJfgCsMHdu2Oz4BGKX,0.036518,0.565093,0.864032,Papi’s Home,3TVXtAsR1Inumwj472S9r4,rap,3SpBlxme9WbeQdI9kx7KAV,0.701,0.741,2,1,0.313,0.112,0.000000,0.699,0.588,4
37Nqx7iavZpotJSDXZWbJ3,0.045568,0.350621,0.822806,Girls Want Girls (with Lil Baby),3TVXtAsR1Inumwj472S9r4,rap,3SpBlxme9WbeQdI9kx7KAV,0.767,0.438,7,0,0.290,0.181,0.000000,0.142,0.381,4
61S79KIVA4I9FXbnsylEHT,0.061137,0.576756,0.828840,In The Bible (with Lil Durk & Giveon),3TVXtAsR1Inumwj472S9r4,rap,3SpBlxme9WbeQdI9kx7KAV,0.686,0.385,7,0,0.297,0.614,0.000000,0.131,0.147,4
4VCbgIdr8ptegWeJpqLVHH,0.046921,0.371406,0.875505,Love All (with JAY-Z),3TVXtAsR1Inumwj472S9r4,rap,3SpBlxme9WbeQdI9kx7KAV,0.730,0.576,2,1,0.287,0.354,0.000036,0.150,0.155,4


In [199]:
df_tracks1.shape

(83698, 17)

In [200]:
# df_tracks1.to_csv('Data\Genre\Tracks_classification_raw.csv')